In [1]:
! which python

/home/cse291project/anaconda3/bin/python


In [11]:
# Copyright (c) 2020-present, Royal Bank of Canada.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

import torch
import numpy as np

from collections import defaultdict, OrderedDict

class VocabEntry(object):
    def __init__(self, vocab_size=100000):
        super(VocabEntry, self).__init__()
        self.vocab_size = vocab_size

        self.word2id = OrderedDict()
        self.unk_id = 3
        self.word2id['<pad>'] = 0
        self.word2id['<s>'] = 1
        self.word2id['</s>'] = 2
        self.word2id['<unk>'] = self.unk_id
        self.id2word_ = list(self.word2id.keys())

    def create_glove_embed(self, glove_file="data/glove.840B.300d.txt"):
        self.glove_embed = np.random.randn(len(self) - 4, 300)
        with open(glove_file) as f:
            for line in f:
                word, vec = line.split(' ', 1)

                wid = self[word]
                if wid > self.unk_id:
                    v = np.fromstring(vec, sep=" ", dtype=np.float32)
                    self.glove_embed[wid - 4, :] = v

        _mu = self.glove_embed.mean()
        _std = self.glove_embed.std()
        self.glove_embed = np.vstack([np.random.randn(4, self.glove_embed.shape[1]) * _std + _mu,
                                      self.glove_embed])

    def __getitem__(self, word):
        idx = self.word2id.get(word, self.unk_id)
        return idx if idx < self.vocab_size else self.unk_id

    def __contains__(self, word):
        return word in self.word2id

    def __len__(self):
        return min(len(self.word2id), self.vocab_size)

    def id2word(self, wid):
        return self.id2word_[wid]

    def decode_sentence(self, sentence):
        decoded_sentence = []
        for wid_t in sentence:
            wid = wid_t.item()
            decoded_sentence.append(self.id2word_[wid])
        return decoded_sentence

    def build(self, sents):
        wordcount = defaultdict(int)
        for sent in sents:
            for w in sent:
                wordcount[w] += 1
        sorted_words = sorted(wordcount, key=wordcount.get, reverse=True)

        for idx, word in enumerate(sorted_words):
            self.word2id[word] = idx + 4
        self.id2word_ = list(self.word2id.keys())

In [52]:
class MonoTextData(object):
    def __init__(self, fname, label=False, max_length=None, vocab=None, glove=False):
        super(MonoTextData, self).__init__()
        self.data, self.vocab, self.dropped, self.sentiment_labels, self.tense_labels = self._read_corpus(
            fname, label, max_length, vocab, glove)

    def __len__(self):
        return len(self.data)

    def _to_tensor(self, batch_data, batch_first, device, min_len=0):
        batch_data = [sent + [self.vocab['</s>']] for sent in batch_data]
        sents_len = [len(sent) for sent in batch_data]
        max_len = max(sents_len)
        max_len = max(min_len, max_len)
        batch_size = len(sents_len)
        sents_new = []
        sents_new.append([self.vocab['<s>']] * batch_size)
        for i in range(max_len):
            sents_new.append([sent[i] if len(sent) > i else self.vocab['<pad>']
                              for sent in batch_data])
        sents_ts = torch.tensor(sents_new, dtype=torch.long,
                                requires_grad=False, device=device)

        if batch_first:
            sents_ts = sents_ts.permute(1, 0).contiguous()

        return sents_ts, [length + 1 for length in sents_len]

    def data_iter(self, batch_size, device, batch_first=False, shuffle=True):
        index_arr = np.arange(len(self.data))
        if shuffle:
            np.random.shuffle(index_arr)
        batch_num = int(np.ceil(len(index_arr)) / float(batch_size))
        for i in range(batch_num):
            batch_ids = index_arr[i * batch_size: (i + 1) * batch_size]
            batch_data = [self.data[index] for index in batch_ids]
            batch_data, sents_len = self._to_tensor(batch_data, batch_first, device)
            yield batch_data, sents_len

    def create_data_batch_labels(self, batch_size, device, batch_first=False, min_len=5):
        sents_len = np.array([len(sent) for sent in self.data])
        sort_idx = np.argsort(sents_len)
        sort_len = sents_len[sort_idx]

        change_loc = []
        for i in range(1, len(sort_len)):
            if sort_len[i] != sort_len[i - 1]:
                change_loc.append(i)
        change_loc.append(len(sort_len))

        batch_data_list = []
        batch_sentiment_label_list = []
        batch_tense_label_list = []
        total = 0
        cur = 0
        for idx in change_loc:
            while cur < idx:
                batch_data = []
                batch_sentiment_label = []
                batch_tense_label =[]
                nxt = min(cur + batch_size, idx)
                for id_ in range(cur, nxt):
                    batch_data.append(self.data[sort_idx[id_]])
                    batch_sentiment_label.append(self.sentiment_labels[sort_idx[id_]])
                    batch_tense_label.append(self.tense_labels[sort_idx[id_]])
                cur = nxt
                batch_data, sents_len = self._to_tensor(batch_data, batch_first, device, min_len)
                batch_data_list.append(batch_data)
                batch_sentiment_label_list.append(batch_sentiment_label)
                batch_tense_label_list.append(batch_tense_label)

                total += batch_data.size(0)
                assert sents_len == ([sents_len[0]] * len(sents_len))

        return batch_data_list, batch_sentiment_label_list, batch_tense_label_list

    def create_data_batch(self, batch_size, device, batch_first=False):
        sents_len = np.array([len(sent) for sent in self.data])
        sort_idx = np.argsort(sents_len)
        sort_len = sents_len[sort_idx]

        change_loc = []
        for i in range(1, len(sort_len)):
            if sort_len[i] != sort_len[i - 1]:
                change_loc.append(i)
        change_loc.append(len(sort_len))

        batch_data_list = []
        total = 0
        cur = 0
        for idx in change_loc:
            while cur < idx:
                batch_data = []
                nxt = min(cur + batch_size, idx)
                for id_ in range(cur, nxt):
                    batch_data.append(self.data[sort_idx[id_]])
                cur = nxt
                batch_data, sents_len = self._to_tensor(batch_data, batch_first, device)
                batch_data_list.append(batch_data)

                total += batch_data.size(0)
                assert sents_len == ([sents_len[0]] * len(sents_len))

        return batch_data_list

    def create_data_batch_feats(self, batch_size, feats, device, batch_first=False):
        sents_len = np.array([len(sent) for sent in self.data])
        print("Maximum length: %d" % max(sents_len))
        sort_idx = np.argsort(sents_len)
        sort_len = sents_len[sort_idx]

        change_loc = []
        for i in range(1, len(sort_len)):
            if sort_len[i] != sort_len[i - 1]:
                change_loc.append(i)
        change_loc.append(len(sort_len))

        batch_data_list = []
        batch_feat_list = []
        total = 0
        cur = 0
        for idx in change_loc:
            while cur < idx:
                batch_data = []
                batch_feat = []
                nxt = min(cur + batch_size, idx)
                for id_ in range(cur, nxt):
                    batch_data.append(self.data[sort_idx[id_]])
                    batch_feat.append(feats[sort_idx[id_]])
                cur = nxt
                batch_data, sents_len = self._to_tensor(batch_data, batch_first, device)
                batch_data_list.append(batch_data)
                batch_feat = torch.tensor(
                    batch_feat, dtype=torch.float, requires_grad=False, device=device)
                batch_feat_list.append(batch_feat)

                total += batch_data.size(0)
                assert sents_len == ([sents_len[0]] * len(sents_len))

        return batch_data_list, batch_feat_list

    def data_sample(self, nsample, device, batch_first=False, shuffle=True):
        index_arr = np.arange(len(self.data))
        if shuffle:
            np.random.shuffle(index_arr)
        batch_ids = index_arr[:nsample]
        batch_data = [self.data[index] for index in batch_ids]

        batch_data, sents_len = self._to_tensor(batch_data, batch_first, device)

        return batch_data, sents_len

    def _read_corpus(self, fname, label, max_length, vocab, glove):
        data = []
        sentiment_labels = [] if label else None
        tense_labels = [] if label else None
        dropped = 0

        sents = []
        with open(fname) as fin:
            header = fin.readline()
            print(header)
            for line in fin:
                if label:
                    split_line = line.strip().split('\t')
                    line_num = split_line[0]
                    sent_lb = split_line[2]
                    tense_lb = split_line[3]
                    split_line = split_line[1].split()
                else:
                    split_line = line.strip().split()

                if len(split_line) < 1:
                    dropped += 1
                    continue

                if max_length:
                    if len(split_line) > max_length:
                        dropped += 1
                        continue

                if label:
                    sentiment_labels.append(int(sent_lb))
                    tense_labels.append(int(tense_lb))
                sents.append(split_line)
                data.append(split_line)

        if isinstance(vocab, int):
            vocab = VocabEntry(vocab)
            vocab.build(sents)
            if glove:
                vocab.create_glove_embed()
        elif vocab is None:
            vocab = VocabEntry()
            vocab.build(sents)
            if glove:
                vocab.create_glove_embed()

        data = [[vocab[word] for word in x] for x in data]

        return data, vocab, dropped, sentiment_labels, tense_labels

In [53]:
train_data_pth = "./data/yelp/train_input_data.csv"
m = MonoTextData(train_data_pth, True)

	content	sentiment-label	tense-label



In [55]:
m.create_data_batch_labels(16, 'cpu')

([tensor([[   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
              1,    1,    1,    1],
          [8976, 6597, 8438, 8976, 8976, 8976, 7349, 6313, 8438, 8976, 8976, 7349,
           8976, 8976, 2769, 2769],
          [   2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
              2,    2,    2,    2],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0]]),
  tensor([[   1,    1],
          [5327, 8976],
          [   2,    2],
          [   0,    0],
          [   0,    0],
          [   0,    0]]),
  tensor([[   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
              1,    1,    1,    1],
          [  60,  5

In [57]:
a = _

In [59]:
x, y, z = a

In [70]:
x[0].shape

torch.Size([6, 16])

In [73]:
dic = dict()

In [74]:
dic["vae_params"] = {
            "lstm_ni": 256,
            "lstm_nh": 1024,
            "lstm_nz": 64,
            "mlp_nz": 16,
            "dec_ni": 128,
            "dec_nh": 1024,
            "dec_dropout_in": 0.5,
            "dec_dropout_out": 0.5,
            "n_vars": 3,
        }

In [80]:
def f (**kwargs):
    print(k`wargs)

In [81]:
f(**dic["vae_params"])

{'lstm_ni': 256, 'lstm_nh': 1024, 'lstm_nz': 64, 'mlp_nz': 16, 'dec_ni': 128, 'dec_nh': 1024, 'dec_dropout_in': 0.5, 'dec_dropout_out': 0.5, 'n_vars': 3}


In [1]:
# import pickle
# train = pickle.load(open("data/train.pkl", "rb"))
# train_sentiments = pickle.load(open("data/dev.pkl", "rb"))
# train_tenses = pickle.load(open("data/test.pkl", "rb"))
# dev = pickle.load(open("data/train_sentiments.pkl", "rb"))
# dev_sentiments = pickle.load(open("data/dev_sentiments.pkl", "rb"))
# dev_tenses = pickle.load(open("data/test_sentiments.pkl", "rb"))
# test = pickle.load(open("data/train_tenses.pkl", "rb"))
# test_sentiments = pickle.load(open("data/dev_tenses.pkl", "rb"))
# test_tenses = pickle.load(open("data/test_tenses.pkl", "rb"))
# feat = train

In [36]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [37]:
# parser = argparse.ArgumentParser()
# add_args(parser)
args = Namespace(data_name="yelp", feat = 'glove', debug = 'False', bsz=4, save="checkpoint/ours", text_only=True)

# main(args)

In [38]:
# Copyright (c) 2020-present, Royal Bank of Canada.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

from utils.exp_utils import create_exp_dir
from utils.text_utils import MonoTextData
import argparse
import os
import torch
import time
import config
import numpy as np


# def main(args):
conf = config.CONFIG[args.data_name]
data_pth = "data/%s" % args.data_name
train_data_pth = os.path.join(data_pth, "train_input_data.csv")
train_feat_pth = os.path.join(data_pth, "train_%s.npy" % args.feat)
train_data = MonoTextData(train_data_pth, True)
train_feat = np.load(train_feat_pth)

vocab = train_data.vocab
print('Vocabulary size: %d' % len(vocab))

dev_data_pth = os.path.join(data_pth, "dev_input_data.csv")
dev_feat_pth = os.path.join(data_pth, "dev_%s.npy" % args.feat)
dev_data = MonoTextData(dev_data_pth, True, vocab=vocab)
dev_feat = np.load(dev_feat_pth)
test_data_pth = os.path.join(data_pth, "test_input_data.csv")
test_feat_pth = os.path.join(data_pth, "test_%s.npy" % args.feat)
test_data = MonoTextData(test_data_pth, True, vocab=vocab)
test_feat = np.load(test_feat_pth)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

save_path = '{}-{}-{}'.format(args.save, args.data_name, args.feat)
save_path = os.path.join(save_path, time.strftime("%Y%m%d-%H%M%S"))
scripts_to_save = [
    'run.py', 'models/decomposed_vae.py', 'models/vae.py',
    'models/base_network.py', 'config.py']
logging = create_exp_dir(save_path, scripts_to_save=scripts_to_save,
                         debug=args.debug)

Vocabulary size: 9603
Debug Mode: no experiment dir created


In [39]:

if args.text_only:
#     train, train_sentiments, train_tenses = train_data.create_data_batch_labels(args.bsz, device)
    dev, dev_sentiments, dev_tenses = dev_data.create_data_batch_labels(args.bsz, device)
    test, test_sentiments, test_tenses = test_data.create_data_batch_labels(args.bsz, device)
    feat = dev
else:
#     train = train_data.create_data_batch_feats(args.bsz, train_feat, device)
    dev = dev_data.create_data_batch_feats(args.bsz, dev_feat, device)
    test = test_data.create_data_batch_feats(args.bsz, test_feat, device)
    feat = train_feat


In [40]:
print("data done.")

kwargs = {
    "train": dev,
    "valid": dev,
    "test": test,
    "train_sentiments": train_sentiments,
    "train_tenses" : train_tenses,
    "dev_sentiments": dev_sentiments,
    "dev_tenses": dev_tenses,
    "test_sentiments": test_sentiments,
    "test_tenses": test_tenses,
    "feat": feat,
    "bsz": args.bsz,
    "save_path": save_path,
    "logging": logging,
    "text_only": args.text_only,
}
params = conf["params"]
params["vae_params"]["vocab"] = vocab
params["vae_params"]["device"] = device
params["vae_params"]["text_only"] = args.text_only
params["vae_params"]["mlp_ni"] = train_feat.shape[1]
kwargs = dict(kwargs, **params)

data done.


In [41]:
from models.decomposed_vae import DecomposedVAE
model = DecomposedVAE(**kwargs)
# try:
#     valid_loss = model.fit()
#     logging("val loss : {}".format(valid_loss))
# except KeyboardInterrupt:
#     logging("Exiting from training early")

# test_loss = model.evaluate(model.test_data, model.test_feat)
# logging("test loss: {}".format(test_loss[0]))
# logging("test recon: {}".format(test_loss[1]))
# logging("test kl1: {}".format(test_loss[2]))
# logging("test kl2: {}".format(test_loss[3]))
# logging("test mi1: {}".format(test_loss[4]))
# logging("test mi2: {}".format(test_loss[5]))

Decomposed VAE: 4 checkpoint/ours-yelp-glove/20201202-160952 functools.partial(<function logging at 0x7f69ac0fc040>, log_path=None, log_=False) 2000 100 0.001 1.0 10 0.1 0.35 0.2 1.0 1.0 0.0 False True {'lstm_ni': 256, 'lstm_nh': 1024, 'lstm_nz': 64, 'mlp_nz': 16, 'dec_ni': 128, 'dec_nh': 1024, 'dec_dropout_in': 0.5, 'dec_dropout_out': 0.5, 'n_vars': 3, 'vocab': <utils.text_utils.VocabEntry object at 0x7f69ac099250>, 'device': device(type='cuda'), 'text_only': True, 'mlp_ni': 300}


In [395]:
# model.load("./checkpoint/ours-yelp-glove/20201201-175217")
# model.load("./checkpoint/ours-yelp-glove/20201202-173317")
# model.load("./checkpoint/ours-yelp-glove/20201202-195410")
# model.load("./checkpoint/ours-yelp-glove/20201202-211936")
# model.load("./checkpoint/ours-yelp-glove/20201202-212112")
model.load("./checkpoint/ours-yelp-glove/20201202-195410")

In [396]:
model.evaluate(model.test_data[:10], model.test_feat[:10])

In eval


/home/cse291project/anaconda3/envs/cp-vae/lib/python3.8/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


(16.43019315358755,
 1.7202762397559914,
 11.441159944276553,
 3.268756969555004,
 0.9608706912478885,
 1.104353363449509)

In [397]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [398]:
# text, _ = test_data._to_tensor(
#     test_data.data[idx:_idx], batch_first=False, device=device)
# feat = torch.tensor(test_feat[idx:_idx], dtype=torch.float, requires_grad=False, device=device)
# z1, _ = model.vae.lstm_encoder(text[:min(text.shape[0], 10)])
# ori_z2, _ = model.vae.mlp_encoder(feat)
# tra_z2 = model.vae.mlp_encoder.var_embedding[var_id:var_id + 1, :].expand(
#     _idx - idx, -1)
# texts = model.vae.decoder.beam_search_decode(z1, tra_z2)
# for text in texts:
#     f.write("%d\t%s\n" % (1 - label, " ".join(text[1:-1])))

# ori_z = torch.cat([z1, ori_z2], -1)
# tra_z = torch.cat([z1, tra_z2], -1)
# for i in range(_idx - idx):
#     ori_logps.append(cal_log_density(mus, logvars, ori_z[i:i + 1].cpu()))
#     tra_logps.append(cal_log_density(mus, logvars, tra_z[i:i + 1].cpu()))

# idx = _idx
# step += 1
# if step % 100 == 0:
#     print(step, idx)

In [399]:
x=None

In [400]:
print(test_data.sentiment_labels[:7])
print(test_data.sentiment_labels[-22:-15])

[1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0]


In [401]:
print(test_data.tense_labels[:7])
print(test_data.tense_labels[-22:-15])

[1, 1, 1, 1, 1, 1, 0]
[1, 0, 1, 0, 0, 0, 0]


In [402]:
import pandas as pd

In [403]:
df = pd.DataFrame()

In [404]:
df

""


In [405]:
neg, _ = test_data._to_tensor(test_data.data[:7], batch_first=False, device=device)
neg_feat = neg

z1neg , _ = model.vae.lstm_encoder(neg[:min(neg.shape[0], 10)])

neg_z2, _ = model.vae.mlp_encoder(neg_feat)

texts = model.vae.decoder.beam_search_decode(z1neg, neg_z2)

for text in texts:
    print("%s\n" % (" ".join(text[1:-1])))
texts = [" ".join(text[1:-1]) for text in texts]
df["A"] = texts

the $ _num_ minimum charge to use a credit card .

sorry but i do n't get the rave reviews .

the desserts were very bland .

the cake portion was extremely light and a bit pricey .

it was super dry and had a weird taste .

once again , i have n't figured out why .

consistently slow .



In [406]:
df["Sentiments - A"] = test_data.sentiment_labels[:7]
df["Tenses - A"] = test_data.tense_labels[:7]

In [407]:
pos, _ = test_data._to_tensor(test_data.data[-22:-15], batch_first=False, device=device)
pos_feat = pos

z1pos , _ = model.vae.lstm_encoder(pos[:min(pos.shape[0], 10)])

pos_z2, _ = model.vae.mlp_encoder(pos_feat)

texts = model.vae.decoder.beam_search_decode(z1pos, pos_z2)
for text in texts:
    print("%s\n" % (" ".join(text[1:-1])))
texts = [" ".join(text[1:-1]) for text in texts]
df["B"] = texts

the food as usual , is good .

the staff is very friendly and attentive .

the tiki lounge is so nice !

the food is also amazing .

service , and the ambiance is awesome !

they have a great selection of cookies / pecan cider .

he also makes sure a live dj who takes us airways as working .



In [408]:
df["Sentiments - B "] = test_data.sentiment_labels[-22:-15]
df["Tenses - B"] = test_data.tense_labels[-22:-15]

In [409]:
# df

In [410]:
nz = z1pos.shape[-1]
# batch_size = z1.shape[1]
nz_half = int(nz/2)
print(nz)

64


In [411]:
z11pos, z12pos = torch.split(z1pos, [nz_half, nz_half], dim=-1)

In [412]:
z11neg, z12neg = torch.split(z1neg, [nz_half, nz_half], dim=-1)

In [413]:
posnegz = torch.cat((z11pos, z12neg), dim=1)

In [414]:
negposz = torch.cat((z11neg, z12pos), dim=1)

In [415]:
# ori_z2, _ = model.vae.mlp_encoder(x_feat)

texts = model.vae.decoder.beam_search_decode(posnegz, pos_z2)

for text in texts:
    print("pos past pos content %s\n" % (" ".join(text[1:-1])))
    
texts = [" ".join(text[1:-1]) for text in texts]
df["B: Z2 - Swapped"] = texts

pos past pos content a favorite place to choose from shop .

pos past pos content i 'm just not taking the money .

pos past pos content the blueberry is flavorful , too !

pos past pos content the sushi is very delicious .

pos past pos content however , the bread is amazing too !

pos past pos content do n't not be real exciting knowing anyone 's used someone else .

pos past pos content they have such a good dining vibe too and her attitude ) .



In [416]:
# ori_z2, _ = model.vae.mlp_encoder(x_feat)

texts = model.vae.decoder.beam_search_decode(negposz, pos_z2)

for i, text in enumerate(texts):
    print("neg present pos cont %s\n" % (" ".join(text[1:-1])))
    
texts = [" ".join(text[1:-1]) for text in texts]
df["B: Z1 - Swapped"] = texts

neg present pos cont the portions were decent , yes , is a tad slow .

neg present pos cont rooms are so clean and the service is excellent .

neg present pos cont the bartender was fantastic !

neg present pos cont the salad was ok , they get a pile of salt .

neg present pos cont the service was very informative , and super cool !

neg present pos cont nacho cheese is delicious as are bacon wrapped pie .

neg present pos cont thank you .



In [417]:
# ori_z2, _ = model.vae.mlp_encoder(x_feat)

texts = model.vae.decoder.beam_search_decode(posnegz, neg_z2)

for text in texts:
    print("pos past neg content %s\n" % (" ".join(text[1:-1])))
texts = [" ".join(text[1:-1]) for text in texts]
df["A: Z1 - Swapped"] = texts

pos past neg content a favorite place to choose from shop .

pos past neg content i 'm just not taking the money .

pos past neg content the blueberry is flavorful and flavorful .

pos past neg content the sushi is very delicious .

pos past neg content however , the bread is amazing also .

pos past neg content do n't not be real exciting knowing anyone 's used someone else .

pos past neg content they have such a good dining vibe too and her attitude ) .



In [418]:
# ori_z2, _ = model.vae.mlp_encoder(x_feat)

texts = model.vae.decoder.beam_search_decode(negposz, neg_z2)

for text in texts:
    print("neg present neg content %s\n" % (" ".join(text[1:-1])))
texts = [" ".join(text[1:-1]) for text in texts]
df["A: Z2 - Swapped"] = texts

neg present neg content the portions were decent , yes , is a tad slow .

neg present neg content rooms are so clean and the service is excellent .

neg present neg content the bartender was very attentive .

neg present neg content the salad was ok , they get a pile of salt .

neg present neg content the service was very informative , and super cool .

neg present neg content nacho cheese is delicious as the coke and caramel .

neg present neg content thank you .



In [419]:
df

,A,Sentiments - A,Tenses - A,B,Sentiments - B,Tenses - B,B: Z2 - Swapped,B: Z1 - Swapped,A: Z1 - Swapped,A: Z2 - Swapped
0,the $ _num_ minimum charge to use a credit card .,1,1,"the food as usual , is good .",0,1,a favorite place to choose from shop .,"the portions were decent , yes , is a tad slow .",a favorite place to choose from shop .,"the portions were decent , yes , is a tad slow ."
1,sorry but i do n't get the rave reviews .,1,1,the staff is very friendly and attentive .,0,0,i 'm just not taking the money .,rooms are so clean and the service is excellent .,i 'm just not taking the money .,rooms are so clean and the service is excellent .
2,the desserts were very bland .,1,1,the tiki lounge is so nice !,0,1,"the blueberry is flavorful , too !",the bartender was fantastic !,the blueberry is flavorful and flavorful .,the bartender was very attentive .
3,the cake portion was extremely light and a bit...,1,1,the food is also amazing .,0,0,the sushi is very delicious .,"the salad was ok , they get a pile of salt .",the sushi is very delicious .,"the salad was ok , they get a pile of salt ."
4,it was super dry and had a weird taste .,1,1,"service , and the ambiance is awesome !",0,0,"however , the bread is amazing too !","the service was very informative , and super c...","however , the bread is amazing also .","the service was very informative , and super c..."
5,"once again , i have n't figured out why .",1,1,they have a great selection of cookies / pecan...,0,0,do n't not be real exciting knowing anyone 's ...,nacho cheese is delicious as are bacon wrapped...,do n't not be real exciting knowing anyone 's ...,nacho cheese is delicious as the coke and cara...
6,consistently slow .,1,0,he also makes sure a live dj who takes us airw...,0,0,they have such a good dining vibe too and her ...,thank you .,they have such a good dining vibe too and her ...,thank you .
